In [400]:
%load_ext autoreload
%autoreload 2
import os, sys
os.environ['STREAM_LOG_LEVEL'] = 'ERROR'
os.environ['FILE_LOG_LEVEL'] = 'DEBUG'
os.environ['PROPAGATE_TO_ROOT_LOGGER'] = 'False'
os.environ['PROPAGATE_TO_ROOT_LOGGER'], os.environ['STREAM_LOG_LEVEL']
from trade.assets.Stock import Stock
from trade.assets.Option import Option
from trade.assets.OptionStructure import OptionStructure
from trade.helpers.Context import Context, clear_context
from trade.helpers.helper import (change_to_last_busday, 
                                  is_USholiday, 
                                  is_busday, 
                                  setup_logger, 
                                  generate_option_tick, 
                                  get_option_specifics_from_key,
                                  identify_length,
                                  extract_numeric_value)
from scipy.stats import percentileofscore
from EventDriven.riskmanager import RiskManager
from dbase.DataAPI.ThetaData import (list_contracts, retrieve_openInterest, retrieve_eod_ohlc, retrieve_quote)
from pandas.tseries.offsets import BDay
from itertools import product
import pandas as pd
from copy import deepcopy
from concurrent.futures import ThreadPoolExecutor, as_completed
from pathos.multiprocessing import ProcessingPool as Pool
import numpy as np
import time
chain_cache = {}
close_cache = {}
oi_cache = {}
spot_cache = {}

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [401]:
tick = 'TSLA'
date = '2023-07-24'
start = (pd.to_datetime(date) - BDay(30)).strftime('%Y-%m-%d')
right = 'C'
order_settings = {
            'type': 'spread',
            'specifics': [
                {'direction': 'long', 'rel_strike': 1.0, 'dte': 365, 'moneyness_width': 0.05},
                {'direction': 'short', 'rel_strike': 0.85, 'dte': 365, 'moneyness_width': 0.05} 
            ],
            'name': 'vertical_spread'
        }

In [ ]:
from typing import List, Dict
from abc import ABC, abstractmethod
from pathos.multiprocessing import ProcessingPool as Pool
from pathos.multiprocessing import cpu_count
from pathos.pools import _ProcessPool
from threading import Thread
from functools import partial
from concurrent.futures import ThreadPoolExecutor

shutdown_event = False

def runProcesses(func, OrderedInputs: List[List], run_type: str = 'map') -> List:
    try:

        pool = Pool(12)
        pool.restart()
        if run_type == 'map':
            results = pool.map(func, *OrderedInputs)
        elif run_type == 'amap':
            results = pool.amap(func, *OrderedInputs)
        elif run_type == 'uimap':
            results = pool.uimap(func, *OrderedInputs)
        elif run_type == 'imap':
            results = pool.imap(func, *OrderedInputs)

        else:
            raise ValueError(f'Run type {run_type} not recognized')

    except KeyboardInterrupt as e:

        shutdown_event = True
        shutdown(pool)
        raise

    except Exception as e:
        print('Error occured: ', e)
        shutdown(pool)


    finally:
        pool.close()
        pool.join()

    return results



def shutdown(pool):
    global shutdown_event
    shutdown_event
    shutdown_event = True
    pool.terminate()

In [244]:
def return_closePrice(id, date):
    global close_cache, spot_cache
    cache_key = f"{id}_{date}"
    close_data = close_cache[cache_key]
    close_data = close_data[~close_data.index.duplicated(keep = 'first')]
    close = close_data['Midpoint'][date]
    return close


def load_chain(date, ticker,  print_stderr = False):
        print(date, ticker) if print_stderr else None
        ## Get both calls and puts per moneyness. For 1 Moneyness, both will most be available. If not, if one is False, other True. 
        ## We will need to get two rows. 
        chain_key = f"{date}_{ticker}"
        with Context(end_date = date):
            if chain_key in chain_cache:
                Option_Chain = chain_cache[chain_key]
            else:
                start_time = time.time()
                Stock_obj = Stock(ticker, run_chain = False)
                end_time = time.time()
                print(f"Time taken to get stock object: {end_time-start_time}") if print_stderr else None
                Option_Chain = Stock_obj.option_chain()
                Spot = Stock_obj.spot(ts = False)
                Spot = list(Spot.values())[0]
                Option_Chain['Spot'] = Spot
                Option_Chain['q'] = Stock_obj.div_yield()
                Option_Chain['r'] = Stock_obj.rf_rate
                chain_cache[chain_key] = Option_Chain



In [243]:
def populate_cache(order_candidates, date = '2024-03-12',):

    global close_cache, oi_cache, spot_cache

    tempholder1 = {}
    tempholder2 = {}

    ## Create necessary data structures
    ## Looping through the order candidates to get the necessary data, and organize into a list of lists that will be passed to runProcesses function
    for j, direction in enumerate(order_candidates):
        for i,data in enumerate(order_candidates[direction]):
            data[[ 'exp', 'strike', 'symbol']] = data[[ 'expiration', 'strike', 'ticker']]
            start = (pd.to_datetime(date) - BDay(20)).strftime('%Y-%m-%d')
            data[['end_date', 'start_date']] = date, start
            data['exp'] = data['exp'].dt.strftime('%Y-%m-%d')
            tempholder1[i+j] = (data[['symbol', 'end_date', 'exp', 'right', 'start_date', 'strike']].T.values.tolist())
            tempholder2[i+j] = data[['symbol', 'right', 'exp','strike']].T.values.tolist()

    ## Extending lists, to ensure only one runProcesses call is made, instead of run per side
    for i, data in tempholder1.items():
        if i == 0:
            OrderedList = data
            tickOrderedList = tempholder2[i]
        else:
            for position, vars in enumerate(data):
                OrderedList[position].extend(vars)
            for position, vars in enumerate(tempholder2[i]):
                tickOrderedList[position].extend(vars)

    
    eod_results = (runProcesses(retrieve_eod_ohlc, OrderedList, 'imap'))
    oi_results = (runProcesses(retrieve_openInterest, OrderedList, 'imap'))
    tick_results = (runProcesses(generate_option_tick, tickOrderedList, 'imap'))
    tick_results = list(set(tick_results))


    ## Save to Dictionary Cache
    for tick, eod, oi in zip(tick_results, eod_results, oi_results):
        cache_key = f"{tick}_{date}"
        close_cache[cache_key] = eod
        oi_cache[cache_key] = oi


    ## Test1: Run spot_cache process after close_cache has been populate.
    
    spot_results = list(runProcesses(return_closePrice, [tick_results, [date]*len(tick_results)], 'imap'))   
    for tick, spot in zip(tick_results, spot_results):
        cache_key = f"{tick}_{date}"
        spot_cache[cache_key] = spot


    ## Test2: We will edit the populate spot_cache populate function to make an api call instead of using the cache.



In [374]:
rm = RiskManager(None, None, 1000000)
tick = 'TSLA'
date = '2023-07-24'
start = (pd.to_datetime(date) - BDay(30)).strftime('%Y-%m-%d')
right = 'C'
order_settings = {
            'type': 'spread',
            'specifics': [
                {'direction': 'long', 'rel_strike': 1.0, 'dte': 365, 'moneyness_width': 0.10},
                {'direction': 'short', 'rel_strike': 0.85, 'dte': 365, 'moneyness_width': 0.10} 
            ],
            'name': 'vertical_spread'
        }
er = rm.OrderPicker.get_order(tick, date, 'C', 10, order_settings)
er

KeyboardInterrupt: 

In [4]:
def produce_order_candidates(settings, tick, date, right = 'P'):
    order_candidates = {'long': [], 'short': []}
    for spec in settings['specifics']:
        order_candidates[spec['direction']].append(chain_details(date, tick, spec['dte'], spec['rel_strike'], right,  moneyness_width = spec['moneyness_width']))
    return order_candidates


def liquidity_check(id, date, pass_threshold = 250):
    sample_id = deepcopy(get_option_specifics_from_key(id))
    new_dict_keys = {'ticker': 'symbol', 'exp_date': 'exp', 'strike': 'strike', 'put_call': 'right'}
    transfer_dict = {}
    for k, v in sample_id.items():

        if k in new_dict_keys:
            if k == 'strike':
                transfer_dict[new_dict_keys[k]] = float(sample_id[k])
            else:
                transfer_dict[new_dict_keys[k]] = sample_id[k]

    start = (pd.to_datetime(date) - BDay(10)).strftime('%Y-%m-%d')
    oi_data = retrieve_openInterest(**transfer_dict, end_date=date, start_date=start)
    # print(f'Open Interest > {pass_threshold} for {id}:', oi_data.Open_interest.mean() )
    return oi_data.Open_interest.mean() > pass_threshold


def available_close_check(id, date, threshold = 0.7):
    cache_key = f"{id}_{date}"
    sample_id = deepcopy(get_option_specifics_from_key(id))
    new_dict_keys = {'ticker': 'symbol', 'exp_date': 'exp', 'strike': 'strike', 'put_call': 'right'}
    transfer_dict = {}
    for k, v in sample_id.items():
        if k in new_dict_keys:
            if k == 'strike':
                transfer_dict[new_dict_keys[k]] = float(sample_id[k])
            else:
                transfer_dict[new_dict_keys[k]] = sample_id[k]
    
    if cache_key in close_cache:
        close_data_sample = close_cache[cache_key]
    else:
        start = (pd.to_datetime(date) - BDay(30)).strftime('%Y-%m-%d')
        close_data_sample = retrieve_eod_ohlc(**transfer_dict, start_date=start, end_date=date)
        close_cache[cache_key] = close_data_sample
    close_mask_series = close_data_sample.Close != 0
    return close_mask_series.sum()/len(close_mask_series) > threshold


def get_structure_price(tradeables, direction_index, date, tick, right = 'P'):
    pack_price = {}
    pack_dataframe = pd.DataFrame()
    pack_dataframe['close'] = 0

    for pack_i, pack in enumerate(tradeables):
        pack_close = 0
        for i, id in enumerate(pack):
            if id not in spot_cache:
                
                cache_key = f"{id}_{date}"
                sample_id = deepcopy(get_option_specifics_from_key(id))
                new_dict_keys = {'ticker': 'symbol', 'exp_date': 'exp', 'strike': 'strike', 'put_call': 'right'}
                transfer_dict = {}
                for k, v in sample_id.items():
                    if k in new_dict_keys:
                        if k == 'strike':
                            transfer_dict[new_dict_keys[k]] = float(sample_id[k])
                        else:
                            transfer_dict[new_dict_keys[k]] = sample_id[k]
                start = (pd.to_datetime(date) - BDay(30)).strftime('%Y-%m-%d')
                close_data_sample = retrieve_eod_ohlc(**transfer_dict, start_date=start, end_date=date)
                close_data_sample = close_data_sample[~close_data_sample.index.duplicated(keep = 'first')]
                close = close_data_sample['Midpoint'][date]
                spot_cache[cache_key] = close
            else:
                close = cache_key[id]
            pack_close += close * direction_index[i]
            pack_dataframe.at[pack_i, i] = id

        pack_dataframe.at[pack_i, 'close'] = pack_close
    return pack_dataframe




In [253]:
load_chain(date, 'TSLA')
order_candidates = produce_order_candidates(order_settings, tick, date, right)

In [254]:

populate_cache(order_candidates, date='2024-03-12')

In [402]:
max_close = 5

direction_index = {}
str_direction_index = {}
for indx, v in enumerate(order_settings['specifics']):
    if v['direction'] == 'long':
        str_direction_index[indx] = 'L'
        direction_index[indx] = 1
    elif v['direction'] == 'short':
        str_direction_index[indx] = 'S'
        direction_index[indx] = -1


load_chain(date, 'TSLA')
order_candidates = produce_order_candidates(order_settings, tick, date, right)


populate_cache(order_candidates, date=date)


for direction in order_candidates:
    for i,data in enumerate(order_candidates[direction]):
        data['liquidity_check'] = data.option_id.apply(lambda x: liquidity_check(x, date))
        data = data[data.liquidity_check == True]
        data['available_close_check'] = data.option_id.apply(lambda x: available_close_check(x, date))
        order_candidates[direction][i] = data[data.available_close_check == True] 



## Filter Unique Combinations per leg.
unique_ids = {'long': [], 'short': []}
for direction in order_candidates:
    for i,data in enumerate(order_candidates[direction]):
        unique_ids[direction].append(data[(data.liquidity_check == True) & (data.available_close_check == True)].option_id.unique().tolist())

## Produce Tradeable Combinations
tradeable_ids = list(product(*unique_ids['long'], *unique_ids['short']))
tradeable_ids, unique_ids 

## Keep only unique combinations. Not repeating a contract.
filtered = [t for t in tradeable_ids if len(set(t)) == len(t)]

## Get the price of the structure
## Using List Comprehension to sum the prices of the structure per index
results = [
    (*items, sum([direction_index[i] * spot_cache[f'{item}_{date}'] for i, item in enumerate(items)])) for items in filtered
]

## Convert to DataFrame, and sort by the price of the structure.
return_dataframe = pd.DataFrame(results)
cols = return_dataframe.columns.tolist()
cols[-1] = 'close'
return_dataframe.columns= cols
return_dataframe = return_dataframe[(return_dataframe.close<= max_close)].sort_values('close', ascending = False).head(1)

## Rename the columns to the direction names
return_dataframe.columns = list(str_direction_index.values()) + ['close']
return_order = return_dataframe[list(str_direction_index.values())].to_dict(orient = 'list')
return_order

## Create the trade_id with the direction and the id of the contract.
id = ''
for k, v in return_order.items():
    id += f"&{k}:{v[0]}"

return_order['trade_id'] = id
return_order['close'] = return_dataframe.close.values[0]

return_order

{'L': ['TSLA20240621000260C'],
 'S': ['TSLA20240621032667C'],
 'trade_id': '&L:TSLA20240621000260C&S:TSLA20240621032667C',
 'close': 3.6499999999999915}

In [399]:
len(spot_cache)

15

In [394]:
order_candidates['long'][0]

right,expiration,DTE,strike,Spot,q,r,relative_moneyness,moneyness_spread,dte_spread,ticker,moneyness,TGT_DTE,right,option_id
build_date,,,,,,,,,,,,,,
2023-07-24,2024-06-21,332,260.00,269.059998,0,0.0525,1.034846,0.001214,1089,TSLA,1.0,365,C,TSLA20240621000260C
2023-07-24,2024-06-21,332,266.67,269.059998,0,0.0525,1.008962,0.000080,1089,TSLA,1.0,365,C,TSLA20240621026667C
2023-07-24,2024-06-21,332,270.00,269.059998,0,0.0525,0.996519,0.000012,1089,TSLA,1.0,365,C,TSLA20240621000270C
2023-07-24,2024-06-21,332,273.33,269.059998,0,0.0525,0.984378,0.000244,1089,TSLA,1.0,365,C,TSLA20240621027333C
2023-07-24,2024-06-21,332,276.67,269.059998,0,0.0525,0.972494,0.000757,1089,TSLA,1.0,365,C,TSLA20240621027667C
2023-07-24,2024-06-21,332,280.00,269.059998,0,0.0525,0.960929,0.001527,1089,TSLA,1.0,365,C,TSLA20240621000280C


In [342]:
return_dataframe.columns = list(str_direction_index.values()) + ['close']
return_order = return_dataframe[list(str_direction_index.values())].to_dict(orient = 'list')

return_order

id = ''
for k, v in return_order.items():
    id += f"&{k}{v[0]}"

return_order['trade_id'] = id
return_order['close'] = return_dataframe.close.values[0]
return_order

{'L:': ['TSLA20250321000175P'],
 'S:': ['TSLA20250321000140P'],
 'trade_id': '&L:TSLA20250321000175P&S:TSLA20250321000140P',
 'close': 4.75}

In [293]:
results = [
    (*items, sum([direction_index[i] * spot_cache[f'{item}_{date}'] for i, item in enumerate(items)])) for items in filtered
]

results_df = pd.DataFrame(results)
cols = results_df.columns.tolist()
cols[-1] = 'price'
results_df.columns= cols
results_df

,0,1,price
0,TSLA20250321000190P,TSLA20250321000175P,-2.450
1,TSLA20250321000190P,TSLA20250321000170P,6.550
2,TSLA20250321000190P,TSLA20250321000165P,-16.525
3,TSLA20250321000190P,TSLA20250321000160P,6.550
4,TSLA20250321000190P,TSLA20250321000155P,0.000
...,...,...,...
60,TSLA20250321000155P,TSLA20250321000150P,4.500
61,TSLA20250321000155P,TSLA20250321000145P,-5.025
62,TSLA20250321000155P,TSLA20250321000140P,2.300
63,TSLA20250321000155P,TSLA20250321000135P,8.500


In [5]:
def chain_details(date, ticker, tgt_dte, tgt_moneyness, right = 'P', moneyness_width = 0.15, print_stderr = False):
    return_dataframe = pd.DataFrame()
    errors = {}
    if not (is_USholiday(date) and not is_busday(date)):
        try:
            print(date, ticker) if print_stderr else None
            ## Get both calls and puts per moneyness. For 1 Moneyness, both will most be available. If not, if one is False, other True. 
            ## We will need to get two rows. 
            chain_key = f"{date}_{ticker}"
            with Context(end_date = date):
                if chain_key in chain_cache:
                    Option_Chain = chain_cache[chain_key]
                else:
                    start_time = time.time()
                    Stock_obj = Stock(ticker, run_chain = False)
                    end_time = time.time()
                    print(f"Time taken to get stock object: {end_time-start_time}") if print_stderr else None
                    Option_Chain = Stock_obj.option_chain()
                    Spot = Stock_obj.spot(ts = False)
                    Spot = list(Spot.values())[0]
                    Option_Chain['Spot'] = Spot
                    Option_Chain['q'] = Stock_obj.div_yield()
                    Option_Chain['r'] = Stock_obj.rf_rate
                    chain_cache[chain_key] = Option_Chain

                
                Option_Chain_Filtered = Option_Chain[Option_Chain[right.upper()] == True]
                
                
                if right == 'P':
                    Option_Chain_Filtered['relative_moneyness']  = Option_Chain_Filtered.index.get_level_values('strike')/Option_Chain_Filtered.Spot
                elif right == 'C':
                    Option_Chain_Filtered['relative_moneyness']  = Option_Chain_Filtered.Spot/Option_Chain_Filtered.index.get_level_values('strike')
                else:
                    raise ValueError(f'Right dne. recieved {right}')
                Option_Chain_Filtered['moneyness_spread'] = (tgt_moneyness-Option_Chain_Filtered['relative_moneyness'])**2
                Option_Chain_Filtered['dte_spread'] = (Option_Chain_Filtered.index.get_level_values('DTE')-tgt_dte)**2
                Option_Chain_Filtered.sort_values(by=['dte_spread','moneyness_spread'], inplace = True)
                Option_Chain_Filtered = Option_Chain_Filtered.loc[Option_Chain_Filtered['dte_spread'] == Option_Chain_Filtered['dte_spread'].min()]
                if float(moneyness_width) == 0.0:
                    option_details = Option_Chain_Filtered.sort_values('moneyness_spread', ascending=False).head(1)
                else:
                    option_details = Option_Chain_Filtered[(Option_Chain_Filtered['relative_moneyness'] >= tgt_moneyness-moneyness_width) & 
                                                        (Option_Chain_Filtered['relative_moneyness'] <= tgt_moneyness+moneyness_width)]
                option_details['build_date'] = date
                option_details['ticker'] = ticker
                option_details['moneyness'] = tgt_moneyness
                option_details['TGT_DTE'] = tgt_dte
                option_details.reset_index(inplace = True)
                option_details.set_index('build_date', inplace = True)
                option_details['right'] = right
                option_details.drop(columns = ['C','P'], inplace = True)
                option_details['option_id'] = option_details.apply(lambda x: generate_option_tick(symbol = x['ticker'], 
                                                                    exp = x['expiration'].strftime('%Y-%m-%d'), strike = float(x['strike']), right = x['right']), axis = 1)
                return_dataframe = pd.concat([return_dataframe, option_details])
            clear_context()
            return_dataframe.drop_duplicates(inplace = True)

        except Exception as e:
            raise

        return return_dataframe.sort_values('relative_moneyness', ascending=False)
    else:
        return None, errors
    

details= chain_details('2024-03-12', 'TSLA', 365, 0.7, moneyness_width = 0.00)

In [6]:
class OrderPicker:
    def __init__(self):
        self.liquidity_threshold = 250
        self.data_availability_threshold = 0.7
        self.lookback = 30

    def get_order(self, 
                  tick, 
                  date,
                  right, 
                  max_close,
                  order_settings):
        
        ## Create necessary data structures
        direction_index = {}
        for indx, v in enumerate(order_settings['specifics']):
            if v['direction'] == 'long':
                direction_index[indx] = 1
            elif v['direction'] == 'short':
                direction_index[indx] = -1

        ## Produce Order Candidates
        start = (pd.to_datetime(date) - BDay(30)).strftime('%Y-%m-%d')
        order_candidates = produce_order_candidates(order_settings, tick, date, right)

        ## Check Liquidity and Close Availability, Filter out those that don't meet the criteria
        for direction in order_candidates:
            for i,data in enumerate(order_candidates[direction]):
                data['liquidity_check'] = data.option_id.apply(lambda x: liquidity_check(x, date))
                order_candidates[direction][i] = data[data.liquidity_check == True]

        for direction in order_candidates:
            for i,data in enumerate(order_candidates[direction]):
                data['available_close_check'] = data.option_id.apply(lambda x: available_close_check(x, date))
                order_candidates[direction][i] = data[data.available_close_check == True] 

        ## Filter Unique Combinations per leg.
        unique_ids = {'long': [], 'short': []}
        for direction in order_candidates:
            for i,data in enumerate(order_candidates[direction]):
                unique_ids[direction].append(data[(data.liquidity_check == True) & (data.available_close_check == True)].option_id.unique().tolist())
        
        ## Produce Tradeable Combinations
        tradeable_ids = list(product(*unique_ids['long'], *unique_ids['short']))
        tradeable_ids, unique_ids 
        
        ## Keep only unique combinations. Not repeating a contract.
        filtered = [t for t in tradeable_ids if len(set(t)) == len(t)]

        ## Get the price of the structure
        prices = get_structure_price(filtered, direction_index, date, 'AAPL')

        ## Return the structure with the best price
        return_dataframe = prices[(prices.close<= max_close)].sort_values('close', ascending = False).head(1)
        return_order = {'long': [], 'short': []}
        id = ''
        for key, v in direction_index.items():
            if v < 0:
                option_id = return_dataframe[key].values[0]
                id += f'&L:{option_id}'
                return_order['short'].append(option_id)
            elif v > 0:
                option_id = return_dataframe[key].values[0]
                id += f'&S:{option_id}'
                return_order['long'].append(option_id)
        
        return_order['trade_id'] = id
        return return_order


order_settings = {
            'type': 'spread',
            'specifics': [
                {'direction': 'long', 'rel_strike': 1.0, 'dte': 365, 'moneyness_width': 0.15},
                {'direction': 'short', 'rel_strike': 0.85, 'dte': 365, 'moneyness_width': 0.15} 
            ],
            'name': 'vertical_spread'
        }


tick = 'TSLA'
date = '2024-03-12'
start = (pd.to_datetime(date) - BDay(30)).strftime('%Y-%m-%d')
right = 'P'


picker = OrderPicker()
er = picker.get_order(tick, date, 'C', 10, order_settings)

In [7]:
class RiskManager:
    def __init__(self,
                 bars,
                 events,
                 initial_capital,
                 ):
        self.bars = bars
        self.events = events
        self.initial_capital = initial_capital
        # self.symbol_list = self.bars.symbol_list
        self.OrderSelector = OrderPicker()


    def get_order(self, symbol, date, order_settings):
        pass



# rm = RiskManager(None, None, 1000000)
# rm.OrderSelector.get_order(tick, date, 'C', 10, order_settings)

In [92]:
import aiohttp
import asyncio
import time

async def fetch_data(session, url):
    print(f"Starting task: {url}")
    async with session.get(url) as response:
        data = await response.json()
        print(f"Completed task: {url}")
        return data

async def main():
    urls = [
        'https://api.github.com',
        'https://api.spacexdata.com/v4/launches/latest',
        'https://jsonplaceholder.typicode.com/todos/1'
    ]
    
    async with aiohttp.ClientSession() as session:
        tasks = [fetch_data(session, url) for url in urls]
        results = await asyncio.gather(*tasks)
        
        for result in results:
            print(result)

if __name__ == '__main__':
    start_time = time.time()
    asyncio.run(main())
    print(f"Total time taken: {time.time() - start_time} seconds")


Starting task: https://api.github.com
Starting task: https://api.spacexdata.com/v4/launches/latest
Starting task: https://jsonplaceholder.typicode.com/todos/1
Completed task: https://api.github.com
Completed task: https://api.spacexdata.com/v4/launches/latest
Completed task: https://jsonplaceholder.typicode.com/todos/1
{'current_user_url': 'https://api.github.com/user', 'current_user_authorizations_html_url': 'https://github.com/settings/connections/applications{/client_id}', 'authorizations_url': 'https://api.github.com/authorizations', 'code_search_url': 'https://api.github.com/search/code?q={query}{&page,per_page,sort,order}', 'commit_search_url': 'https://api.github.com/search/commits?q={query}{&page,per_page,sort,order}', 'emails_url': 'https://api.github.com/user/emails', 'emojis_url': 'https://api.github.com/emojis', 'events_url': 'https://api.github.com/events', 'feeds_url': 'https://api.github.com/feeds', 'followers_url': 'https://api.github.com/user/followers', 'following_url

In [107]:
df = order_candidates['long'][0].copy()
df[[ 'exp', 'strike', 'symbol']] = df[[ 'expiration', 'strike', 'ticker']]
start = (pd.to_datetime(date) - BDay(20)).strftime('%Y-%m-%d')
df[['end_date', 'start_date']] = date, start
df['exp'] = df['exp'].dt.strftime('%Y-%m-%d')
tick_OrderedList = df[['symbol', 'right', 'exp','strike']].T.values
OrderedList = df[['symbol', 'end_date', 'exp', 'right', 'start_date', 'strike']].T.values
results = runProcesses(retrieve_openInterest, OrderedList, 'imap')
oi_results = runProcesses(retrieve_openInterest, OrderedList, 'imap')
tick_results = runProcesses(generate_option_tick, tick_OrderedList, 'imap')
list(results), list(oi_results), list(tick_results)

([    Open_interest      Date      time   Datetime
  0             147  20240213  06:30:00 2024-02-13
  1             147  20240214  06:30:00 2024-02-14
  2             162  20240215  06:30:00 2024-02-15
  3             161  20240216  06:30:01 2024-02-16
  4             165  20240220  06:30:00 2024-02-20
  5             166  20240221  06:30:01 2024-02-21
  6             166  20240222  06:30:00 2024-02-22
  7             164  20240223  06:30:01 2024-02-23
  8             163  20240226  06:30:00 2024-02-26
  9             181  20240227  06:30:00 2024-02-27
  10            172  20240228  06:30:01 2024-02-28
  11            191  20240229  06:30:00 2024-02-29
  12            157  20240301  06:30:00 2024-03-01
  13            159  20240304  06:30:00 2024-03-04
  14            165  20240305  06:30:00 2024-03-05
  15            174  20240306  06:30:00 2024-03-06
  16            207  20240307  06:30:01 2024-03-07
  17            207  20240308  06:30:00 2024-03-08
  18            209  20240311  

In [13]:
order_candidates['long'][0]

right,expiration,DTE,strike,Spot,q,r,relative_moneyness,moneyness_spread,dte_spread,ticker,moneyness,TGT_DTE,right,option_id
build_date,,,,,,,,,,,,,,
2024-03-12,2025-03-21,373,190.0,173.229996,0.005542,0.0524,1.096808,0.009372,64,AAPL,1.0,365,P,AAPL20250321000190P
2024-03-12,2025-03-21,373,185.0,173.229996,0.005542,0.0524,1.067944,0.004616,64,AAPL,1.0,365,P,AAPL20250321000185P
2024-03-12,2025-03-21,373,180.0,173.229996,0.005542,0.0524,1.039081,0.001527,64,AAPL,1.0,365,P,AAPL20250321000180P
2024-03-12,2025-03-21,373,175.0,173.229996,0.005542,0.0524,1.010218,0.000104,64,AAPL,1.0,365,P,AAPL20250321000175P
2024-03-12,2025-03-21,373,170.0,173.229996,0.005542,0.0524,0.981354,0.000348,64,AAPL,1.0,365,P,AAPL20250321000170P
2024-03-12,2025-03-21,373,165.0,173.229996,0.005542,0.0524,0.952491,0.002257,64,AAPL,1.0,365,P,AAPL20250321000165P
2024-03-12,2025-03-21,373,160.0,173.229996,0.005542,0.0524,0.923628,0.005833,64,AAPL,1.0,365,P,AAPL20250321000160P


In [ ]:
order_settings = {
            'type': 'spread',
            'specifics': [
                {'direction': 'long', 'rel_strike': 1.0, 'dte': 365, 'moneyness_width': 0.1},
                {'direction': 'short', 'rel_strike': 0.85, 'dte': 365, 'moneyness_width': 0.1} 
            ],
            'name': 'vertical_spread'
        }


tick = 'TSLA'
date = '2023-07-24'
start = (pd.to_datetime(date) - BDay(30)).strftime('%Y-%m-%d')
right = 'C'

direction_index = {}
for indx, v in enumerate(order_settings['specifics']):
    if v['direction'] == 'long':
        direction_index[indx] = 1
    elif v['direction'] == 'short':
        direction_index[indx] = -1

direction_index


{0: 1, 1: -1}

In [42]:
# for direction in order_candidates:
#     for i,data in enumerate(order_candidates[direction]):
#         data['liquidity_check'] = data.option_id.apply(lambda x: liquidity_check(x, date))
#         data['available_close_check'] = data.option_id.apply(lambda x: available_close_check(x, date))
#         order_candidates[direction][i] = data[data.liquidity_check == True]

# for direction in order_candidates:
#     for i,data in enumerate(order_candidates[direction]):
#         data['available_close_check'] = data.option_id.apply(lambda x: available_close_check(x, date))
#         order_candidates[direction][i] = data[data.available_close_check == True]  


for direction in order_candidates:
    for i,data in enumerate(order_candidates[direction]):
        data['liquidity_check'] = data.option_id.apply(lambda x: liquidity_check(x, date))
        data['available_close_check'] = data.option_id.apply(lambda x: available_close_check(x, date))


## Filter Unique Combinations per leg.

unique_ids = {'long': [], 'short': []}
for direction in order_candidates:
    for i,data in enumerate(order_candidates[direction]):
        unique_ids[direction].append(data.option_id.unique())

tradeable_ids = list(product(*unique_ids['long'], *unique_ids['short']))
  

In [44]:
filtered = [t for t in tradeable_ids if len(set(t)) == len(t)]
pd.DataFrame(filtered)
filtered

[('TSLA20240621024667C', 'TSLA20240621028333C'),
 ('TSLA20240621024667C', 'TSLA20240621028667C'),
 ('TSLA20240621024667C', 'TSLA20240621000290C'),
 ('TSLA20240621024667C', 'TSLA20240621029333C'),
 ('TSLA20240621024667C', 'TSLA20240621029667C'),
 ('TSLA20240621024667C', 'TSLA20240621000300C'),
 ('TSLA20240621024667C', 'TSLA20240621030333C'),
 ('TSLA20240621024667C', 'TSLA20240621030667C'),
 ('TSLA20240621024667C', 'TSLA20240621000310C'),
 ('TSLA20240621024667C', 'TSLA20240621031667C'),
 ('TSLA20240621024667C', 'TSLA20240621000320C'),
 ('TSLA20240621024667C', 'TSLA20240621032667C'),
 ('TSLA20240621024667C', 'TSLA20240621000330C'),
 ('TSLA20240621024667C', 'TSLA20240621033333C'),
 ('TSLA20240621024667C', 'TSLA20240621033667C'),
 ('TSLA20240621024667C', 'TSLA20240621000340C'),
 ('TSLA20240621024667C', 'TSLA20240621034333C'),
 ('TSLA20240621024667C', 'TSLA20240621034667C'),
 ('TSLA20240621024667C', 'TSLA20240621000350C'),
 ('TSLA20240621024667C', 'TSLA20240621035333C'),
 ('TSLA2024062102466

In [ ]:
prices = get_structure_price(filtered, direction_index, date, 'AAPL')
prices


In [45]:
return_dataframe = prices[(prices.close<= 5) & (prices.close>0)].sort_values('close', ascending = False).head(1)
return_order = {'long': [], 'short': []}
for key, v in direction_index.items():
    if v < 0:
        return_order['short'].append(return_dataframe[key].values[0])
    elif v > 0:
        return_order['long'].append(return_dataframe[key].values[0])
return_order

{'long': ['AAPL20250620000230C'], 'short': ['AAPL20250620000240C']}

In [37]:
prices#[prices.close<= 5]
prices#[(prices.close<= 5) & (prices.close>0)].sort_values('close', ascending = False)

,0,1,close
0,AAPL20250321000180P,AAPL20250321000160P,7.95
1,AAPL20250321000180P,AAPL20250321000150P,10.55
2,AAPL20250321000180P,AAPL20250321000140P,12.40
3,AAPL20250321000175P,AAPL20250321000160P,5.55
4,AAPL20250321000175P,AAPL20250321000150P,8.15
5,AAPL20250321000175P,AAPL20250321000140P,10.00
6,AAPL20250321000170P,AAPL20250321000160P,3.45
7,AAPL20250321000170P,AAPL20250321000150P,6.05
8,AAPL20250321000170P,AAPL20250321000140P,7.90
9,AAPL20250321000165P,AAPL20250321000160P,1.60


In [20]:
prices[prices.close<= 5]

,0,1,close
6,AAPL20250321000170P,AAPL20250321000160P,3.45
9,AAPL20250321000165P,AAPL20250321000160P,1.60
10,AAPL20250321000165P,AAPL20250321000150P,4.20
12,AAPL20250321000160P,AAPL20250321000150P,2.60
13,AAPL20250321000160P,AAPL20250321000140P,4.45


In [21]:
min(prices.values())

TypeError: 'numpy.ndarray' object is not callable

In [275]:
order_candidates['long'][0]

right,expiration,DTE,strike,relative_moneyness,moneyness_spread,dte_spread,ticker,Spot,moneyness,TGT_DTE,q,r,right,option_id,liquidity_check,available_close_check
build_date,,,,,,,,,,,,,,,,
2024-03-12,2025-03-21,373,180.0,1.039081,0.001527,64,AAPL,173.229996,1.0,365,0.005542,0.0524,P,AAPL20250321000180P,True,True
2024-03-12,2025-03-21,373,170.0,0.981354,0.000348,64,AAPL,173.229996,1.0,365,0.005542,0.0524,P,AAPL20250321000170P,True,True
2024-03-12,2025-03-21,373,165.0,0.952491,0.002257,64,AAPL,173.229996,1.0,365,0.005542,0.0524,P,AAPL20250321000165P,True,True


In [277]:
order_candidates['short'][0]

right,expiration,DTE,strike,relative_moneyness,moneyness_spread,dte_spread,ticker,Spot,moneyness,TGT_DTE,q,r,right,option_id,liquidity_check,available_close_check
build_date,,,,,,,,,,,,,,,,
2024-03-12,2025-03-21,373,150.0,0.865901,0.000253,64,AAPL,173.229996,0.85,365,0.005542,0.0524,P,AAPL20250321000150P,True,True


Steps to producing an order:

- S1: RM recieves order settings from PM
- S2: RM produces a dataframe of potential options based on settings (if two legs produce two dataframes)
- S3: RM assesses if option passes all checks
    - C1: Minimum Available close
    - C2: Liquidity (Open Interest)
    - C2.5: (for Spreads only) Ensure both legs are not the same
    - Optional, to extend:
    - C3: Bid-Ask Spread
    
- S4: Return picked order to portfolio manager, which places the order. 
- Example:
    {'long': [optionid or {'strike', 'exp'}], 'short' : []}

In [65]:
generate_option_tick('AAPL','P' ,'2025-03-21', 150.0)
# save_option_keys(generate_option_tick('AAPL','P' ,'2025-03-21', 150.0), {'ticker':'AAPL', 'put_call':'P', 'exp_date':'2025-03-21', 'strike':150.0})

'AAPL20250321000150P'

In [256]:
opt = Option(**option_keys['AAPL20250321000150P'], run_chain = True)

NameError: name 'option_keys' is not defined

In [39]:
import_option_keys()
option_keys

{}